In [52]:
import osmnx as ox
import networkx as nx
import sklearn
import numpy as np
import folium
from folium import plugins
from classes import person, visits, team
import in_out

In [3]:
# Populate tracked instances list

_class_list = (person.Patient, person.Clinician, visits.Visit, team.Team)

for cls in _class_list:
    cls.load_tracked_instances()

In [4]:
def display_route(coord_list):
    """
    Uses Folium to display a route given a list of coordinates
    :param coord_list: List of tuples containing coordinate information
    :param visit_list: Ordered list of visit objects to use for generating markers and the tooltip
    :return: Ordered list of appointment coordinates for optimal travel time
    """
    # Calculate central point to initialize map
    center_coord = coord_average(coord_list)

    # Initialize map and set boundaries.
    map = folium.Map(location=center_coord)
    map.fit_bounds(coord_list)

    color_list = list(folium.Icon.color_options)

    # Loop through each coord and create a new marker and tooltip
    for index, coord in enumerate(coord_list):
        tooltip = f"<center><h2>{index}. {coord}</h2></center>" \

        folium.Marker(
            coord,
            tooltip=tooltip,
            icon=folium.Icon(color=color_list[index], icon=f"circle-{index}")
        ).add_to(map)

    map.render()

def coord_average(coord_list):
    """
    Finds the average point between all coordinates in the coordinate list in order to center the map.
    :param coord_list: List of tuples containing coordinate information
    :return: Tuple of average longitude and latitude
    """
    # Split latitude and longitude
    lat_list = np.array([coord[0] for coord_group in coord_list for coord in coord_group])
    long_list = np.array([coord[1] for coord_group in coord_list for coord in coord_group])
    
    # Average latitude and longitude
    mean_lat = lat_list.mean()
    mean_long = long_list.mean()

    return mean_lat, mean_long

In [5]:
team_obj = in_out.load_obj(team, f"./data/Team/{10000}.pkl")
date = "20/12/2022"

clins = [in_out.load_obj(person.Clinician, f"./data/Clinician/{clin_id}.pkl") for clin_id in team_obj._clin_id]
visit_ids_by_clin = [clin.visits[date] for clin in clins]
visit_ids_by_clin

[[10003, 10000, 10007, 10001, 10019, 10008],
 [10005, 10006, 10018, 10004, 10011],
 [10015, 10017, 10009, 10013, 10002, 10016],
 [10020, 10014, 10010, 10012]]

In [6]:
visit_list = [[in_out.load_obj(visits.Visit, f"./data/Visit/{visit_id}.pkl") for visit_id in visit_ids] for visit_ids in visit_ids_by_clin]
coord_list = [[visit.coord for visit in visit_group] for visit_group in visit_list]
coord_list

[[(47.6121162, -122.3430024),
  (47.5933101, -122.3322722),
  (47.6252232, -122.3154426),
  (47.7325024, -122.3548937),
  (47.6012584, -122.3296063),
  (47.602524, -122.3041695)],
 [(47.6012584, -122.3296063),
  (47.6012584, -122.3296063),
  (47.5933101, -122.3322722),
  (47.5933101, -122.3322722),
  (47.576062, -122.4090846)],
 [(47.7325024, -122.3548937),
  (47.6121162, -122.3430024),
  (47.7265746, -122.3481006),
  (47.6252232, -122.3154426),
  (47.53426, -122.3461206),
  (47.53426, -122.3461206)],
 [(47.6252232, -122.3154426),
  (47.6141665, -122.3425843),
  (47.5606467, -122.2923036),
  (47.6252232, -122.3154426)]]

In [7]:
coord_avr = coord_average(coord_list)
coord_avr

(47.61534901428572, -122.33484199523812)

In [8]:
def number_DivIcon(color,number):
    """ Create a 'numbered' icon
    
    """
    icon = folium.features.DivIcon(
            icon_size=(150,36),
            icon_anchor=(18,38),
            html="""<span class="fa-stack" style="font-size: 12pt">
                    <!-- The icon that will wrap the number -->
                    <span class="fa fa-circle-o fa-stack-2x" style="color:{:s}"></span>
                    <!-- a strong element with the custom content, in this case a number -->
                    <strong class="fa-stack-1x">{:02d}</strong>
                </span>""".format(color,number)
        )
    return icon

In [9]:
%matplotlib inline
# Calculate central point to initialize map
center_coord = coord_average(coord_list)

# Initialize map and set boundaries.
map = folium.Map(location=center_coord)
map.fit_bounds(coord_list)

color_list = ('darkblue', 'purple', 'orange', 'green', 'beige', 'lightgreen', 'blue', 'pink', 'lightred', 'red', 'lightgray')

# Loop through each coord group by clinician and create a new marker and tooltip, assorted by color for each clinician
for clin_index, coord_group in enumerate(coord_list):
    # Add feature group for each clinician so they can be turned on or off
    feature_group = folium.FeatureGroup(name=f"{clins[clin_index].name}").add_to(map)
    
    # Create markers for feature group
    for visit_index, coord in enumerate(coord_group):
        tooltip = f"""
                    <center><h4>{visit_index+1}. {visit_list[clin_index][visit_index].patient_name}</h4></center>
                    <p><b>Address</b>: {visit_list[clin_index][visit_index].address}</p>
                    <p><b>Time Window</b>: {visit_list[clin_index][visit_index].time_earliest} - {visit_list[clin_index][visit_index].time_latest}</p>
                    <p><b>Priority</b>: {visit_list[clin_index][visit_index].visit_priority}</p>
                    <p><b>Complexity</b>: {visit_list[clin_index][visit_index].visit_complexity}</p>
                    <p><b>Skills Required</b>: {visit_list[clin_index][visit_index].skill_list}</p>
                    <p><b>Discipline Requested</b>: {visit_list[clin_index][visit_index].discipline}</p>
                    """ 

        # Create marker image
        feature_group.add_child(
            folium.Marker(
            coord,
            tooltip=tooltip,
            icon=folium.Icon(icon_color='white', color=color_list[clin_index]),
            ))

        # Add number to marker
        feature_group.add_child(
            folium.Marker(
                coord,
                tooltip=tooltip,
                icon=number_DivIcon(color_list[clin_index], visit_index+1)
            ))
folium.LayerControl().add_to(map)

map

In [10]:
# Plot route on map
# find shortest route based on the mode of travel
mode      = 'drive'        # 'drive', 'bike', 'walk'
# find shortest path based on distance or time
optimizer = 'time'        # 'length','time'
# create graph from OSM using central coordinate
graph = ox.graph_from_point(center_coord, dist=20000, network_type = mode)
ox.settings.log_console=True
ox.settings.use_cache=True

In [57]:
# Calculate central point to initialize map
center_coord = coord_average(coord_list)

# Initialize map and set boundaries.
route_map = folium.Map(location=center_coord)

for clin_index, coord_group in enumerate(coord_list):
    # Create routes for each visit for the clinician
    feature_group = folium.FeatureGroup(name=f"{clins[clin_index].name}").add_to(route_map)
    marker_group = plugins.FeatureGroupSubGroup(feature_group, name=f"Markers - {clins[clin_index].name}").add_to(route_map)
    route_group = plugins.FeatureGroupSubGroup(feature_group, name=f"Route - {clins[clin_index].name}").add_to(route_map)
    # TODO: Add start and end marker for each clinician

    for visit_index in range(len(coord_group)-1):
        # define the start and end locations in latlng
        start_latlng = coord_group[visit_index]
        end_latlng = coord_group[visit_index+1]
        # find the nearest node to the start location
        orig_node = ox.nearest_nodes(graph, start_latlng[1], start_latlng[0])
        # find the nearest node to the end location
        dest_node = ox.nearest_nodes(graph, end_latlng[1], end_latlng[0])
        #  find the shortest path
        shortest_route = nx.shortest_path(graph,
                                        orig_node,
                                        dest_node,
                                        weight=optimizer)
        # Add route to map
        if orig_node != dest_node:
            coords = [(graph.nodes[node]["y"], graph.nodes[node]["x"]) for node in shortest_route]
            route_group.add_child(folium.PolyLine(coords, weight=3, color=color_list[clin_index]))

        tooltip = f"""
                    <center><h4>{visit_index+1}. {visit_list[clin_index][visit_index].patient_name}</h4></center>
                    <p><b>Address</b>: {visit_list[clin_index][visit_index].address}</p>
                    <p><b>Time Window</b>: {visit_list[clin_index][visit_index].time_earliest} - {visit_list[clin_index][visit_index].time_latest}</p>
                    <p><b>Priority</b>: {visit_list[clin_index][visit_index].visit_priority}</p>
                    <p><b>Complexity</b>: {visit_list[clin_index][visit_index].visit_complexity}</p>
                    <p><b>Skills Required</b>: {visit_list[clin_index][visit_index].skill_list}</p>
                    <p><b>Discipline Requested</b>: {visit_list[clin_index][visit_index].discipline}</p>
                    """ 

        # Create marker image
        marker_group.add_child(
            folium.Marker(
            coord_group[visit_index],
            tooltip=tooltip,
            icon=folium.Icon(icon_color='white', color=color_list[clin_index]),
            ))

        # Add number to marker
        marker_group.add_child(
            folium.Marker(
                coord_group[visit_index],
                tooltip=tooltip,
                icon=number_DivIcon(color_list[clin_index], visit_index+1)
            ))
        
folium.LayerControl().add_to(route_map)
route_map
